M00967179 + M009613339

# Hot-Desk Detector - ROCm 6.0 + YOLOv11

#### **Hardware**:
**GPU**: AMD Radeon RX6800XT (16GB VRAM) [Relevant for Batch and Imgsz]  
**CPU**: AMD Ryzen 5 9600X (6Cores/12Threads) [Relevant for amount of Workers]  
**RAM**: Crucial DDR5 Pro 32GB @6400MT/s [Relvant for cache:'ram']

**Model**: YOLOv11m  
**Classes**: 8

### Verify that the environment works and that the GPU is being detected  
**Note**: If GPU is not detected, PyTorch will default to CPU

In [1]:
import torch
import ultralytics
import numpy as np

print(f"🔥 PyTorch: {torch.__version__}")
print(f"🎯 Ultralytics: {ultralytics.__version__}")
print(f"🖥️  GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")
print(f"✅ ROCm available: {torch.cuda.is_available()}")
print(f"📊 CUDA devices: {torch.cuda.device_count()}")

🔥 PyTorch: 2.4.1+rocm6.0
🎯 Ultralytics: 8.3.221
🖥️  GPU: AMD Radeon RX 6800 XT
✅ ROCm available: True
📊 CUDA devices: 1


### Model Initialization  
**Note**: The commented out bit is for the YOLOv8m model, which was used for initial testing.

In [2]:
# from ultralytics import YOLO

# # Load medium-sized YOLOv8 model
# model = YOLO('yolov8m.pt')
# print("✅ YOLOv8m model loaded successfully!")

# # Display model architecture
# print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

# ==========================================
# 3. MODEL INITIALISATION – YOLO11
# ==========================================
from ultralytics import YOLO

model = YOLO("yolo11m.pt")          # Set to yolo11n.pt for way quicker experiments and if on CPU
print("✅ YOLO11m model loaded successfully!")
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

✅ YOLO11m model loaded successfully!
Model parameters: 20,114,688


### Phase 1  
Full Dataset + Heavy Aug

In [ ]:
phase1 = dict(
    data="data/data.yaml",
    epochs=200,
    imgsz=640,                # Lower to 448 if having issues or on CPU.
    batch=32,                 # Recommeded to lower to 8 if on CPU!!
    workers=10,               # 10 out of 12 Threads is being used. DO NOT use all threads, it will cause your OS to crash.
    device=0,                 # device=0 is GPU, change to device='cpu' for CPU.
    optimizer="AdamW",
    lr0=0.001,
    lrf=0.05,
    weight_decay=0.0005,
    momentum=0.937,
    warmup_epochs=3,
    amp=True,                 # ROCm native AMP, set to False for CPU!!
    half=False,               # keep FP32 BN 
    cache="ram",              # 32 GB DDR5. Use cache="disk" if the RAM cannot handle it.
    compile=False,            # If True, training time will be faster, but caused issues on ROCm env so I set to False.       
    patience=20,              # If mAP50 does not improve after 20 epochs, stop training.
    project="hotdesk_training",
    name="yolo11m_phase1",
    exist_ok=True,
    # ----- augmentation -----
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=5.0,
    translate=0.1,
    scale=0.5,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.2,
    copy_paste=0.1,
    perspective=0.0005,
    flipud=0.0,
    # ----- loss -----
    label_smoothing=0.05,
    box=7.5,
    cls=0.5,
    dfl=1.5,
    # ----- outputs -----
    save_period=10,          # Saves model every 10 epoch so that progress is not loss on event of a system crash.
    plots=True,
)

print("🔥 STARTING PHASE-1 – YOLO11m full-training")
results_p1 = model.train(**phase1)

### Phase 2  
Fine-tune best weight  
i.e lower LR, lighter aug, longer epochs.

In [ ]:
# DO NOT FORGET TO SET THE SAME TOGGLE AS ABOVE IF ON CPU!!
from pathlib import Path
model_path = Path('model/hotdest_final_model.pt') # Set this path to wherever the best.pt was generated from the above training.
phase2 = dict(
    data="data/data.yaml",
    epochs=50,                 # enough to drop LR 5×
    imgsz=640,
    batch=16,                  # smaller micro-batch → stabler gradients
    workers=8,                
    device=0,
    optimizer="AdamW",
    lr0=1e-4,                  # 10× lower than phase-1
    lrf=0.01,
    weight_decay=0.0005,
    warmup_epochs=1,
    amp=True,
    half=False,
    cache="ram",
    compile=False,             # keep off until ROCm fix
    patience=20,
    project="hotdesk_training",
    name="yolo11m_phase2_finetune",
    exist_ok=True,
    # ----- LIGHTER AUG → less distortion for mug/bottle -----
    hsv_h=0.01,
    degrees=2.0,
    translate=0.05,
    scale=0.2,
    fliplr=0.5,
    mosaic=0.5,
    mixup=0.0,
    copy_paste=0.0,
    # ----- start from best -----
    model=model_path, 
)

print("🎯 STARTING PHASE-2 – fine-tune best checkpoint")
model2 = YOLO(model_path)
results_p2 = model2.train(**phase2)

### Phase 3  
Micro fine-tune for underperformer: mug

In [ ]:
from ultralytics import YOLO

from pathlib import Path
model_path = Path('model/hotdest_final_model.pt') # Set this path to wherever the best.pt was generated from the above training.

model3 = YOLO(model_path)

# start training – this creates model.trainer but does NOT run epochs yet
model3.train(
    data="data/data.yaml",
    epochs=10,
    imgsz=640,
    batch=16,
    workers=10,
    device=0,
    optimizer="AdamW",
    lr0=5e-5,
    lrf=0.01,
    weight_decay=0.0005,
    warmup_epochs=0,
    amp=True,
    cache="ram",
    compile=False,
    patience=0,
    project="hotdesk_training",
    name="yolo11m_mug_boost",
    exist_ok=True,
    mosaic=0.2,
    mixup=0.0,
    copy_paste=0.8,
    degrees=1.0,
    translate=0.05,
    scale=0.1
)


# ----- and start the actual epoch loop -----
model3.trainer.train()

### Model Evaluation  
Run the model against the test-split (unseen data)

In [ ]:
print("🧪 Comprehensive Test Set Evaluation")
print("=" * 50)

import os
import numpy as np
from ultralytics import YOLO
from pathlib import Path
model_path = Path('model/hotdest_final_model.pt') # Set this path to wherever the best.pt was generated from the above training.

# Load model and test
model = YOLO(model_path)

# Run evaluation with plots
metrics = model.val(
    data='data/data.yaml',
    split='test',
    device=0, # Change to device='cpu' if on CPU!!
    plots=True,
    verbose=True,
    conf=0.15
)

print("🎯 FINAL TEST SET RESULTS:")
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}") 

# Handle array precision/recall values
precision = metrics.box.p
recall = metrics.box.r

# If they're arrays, take the mean
if hasattr(precision, '__iter__'):
    precision_mean = np.mean(precision)
    recall_mean = np.mean(recall)
else:
    precision_mean = precision
    recall_mean = recall

print(f"Precision: {precision_mean:.4f}")
print(f"Recall: {recall_mean:.4f}")

# Calculate F1 Score (Micro-F1)
f1_score = 2 * (precision_mean * recall_mean) / (precision_mean + recall_mean + 1e-16)
print(f"F1 Score (Micro): {f1_score:.4f}")

print("\n📊 PER-CLASS TEST PERFORMANCE:")
class_names = ['mug', 'headset', 'mouse', 'stapler', 'notebook', 'pen', 'phone', 'bottle']
class_f1_scores = []

for i, class_idx in enumerate(metrics.box.ap_class_index):
    class_name = class_names[int(class_idx)]
    ap50 = metrics.box.ap50[i]
    
    # Get per-class precision and recall
    if hasattr(metrics.box.p, '__iter__') and i < len(metrics.box.p):
        class_precision = metrics.box.p[i]
    else:
        class_precision = precision_mean
        
    if hasattr(metrics.box.r, '__iter__') and i < len(metrics.box.r):
        class_recall = metrics.box.r[i]
    else:
        class_recall = recall_mean
    
    # Calculate per-class F1
    class_f1 = 2 * (class_precision * class_recall) / (class_precision + class_recall + 1e-16)
    class_f1_scores.append(class_f1)
    
    print(f"  {class_name}: AP50={ap50:.4f}, Precision={class_precision:.4f}, Recall={class_recall:.4f}, F1={class_f1:.4f}")

# Calculate Macro-F1 (average of per-class F1 scores)
if class_f1_scores:
    macro_f1 = np.mean(class_f1_scores)
    print(f"\n📈 MACRO-F1 SCORE: {macro_f1:.4f}")
    
    # Also show per-class F1 statistics
    print(f"   - Best F1: {np.max(class_f1_scores):.4f}")
    print(f"   - Worst F1: {np.min(class_f1_scores):.4f}")
    print(f"   - F1 Std: {np.std(class_f1_scores):.4f}")
else:
    macro_f1 = 0.0
    print(f"\n📈 MACRO-F1 SCORE: {macro_f1:.4f} (no per-class data available)")

print(f"\n📊 SUMMARY:")
print(f"Micro-F1 (overall): {f1_score:.4f}")
print(f"Macro-F1 (class-average): {macro_f1:.4f}")

print(f"\n📈 Plots saved to: {metrics.save_dir}")